In [ ]:
!pip install pendulum

# 인사이트
## 데이터 시각화
* 시간에 따른 time_col의 변화를 관찰
    * time_col : `'정원', '휴가자', '출장자', '야근자', '재택근무자', '중식계', '석식계'`

* 삼성 SDS에서 실시한 적이 있는 대회라고 함
    * RNN에서 휴가와 날씨에 가중치를 둬서 좋은 결과를 냈다고 함 -> 참고하면 좋을 듯
    * 일기예보 데이터를 사용해보는 건 어떨까?

<br>

### 가설 1. 재택근무는 코로나 때문에 생겼을 것이다.
* 코로나 이전과 이후로, 식수 인원이 큰 차이가 있을 것이다

<br>

### 가설 2. 날씨가 큰 영향을 미칠것이다
* 월별, 계절별 데이터가 차이날 것


<br>

### 궁금증 1. 조식이 과연 의미가 있을까?
* 일단 없을거라 생각되지만 확인하기 어려움
    * 조식 아침 8시 제공
    * 하루 본사 2명 숙직 18-익일09시
    * 점심 1140-1300, 저녁 1800-
        * 1140-1210 정도로 생각하라고 함


### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import Library

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import datetime
np.random.seed(0)

from sklearn.metrics import roc_auc_score, log_loss

from tqdm.notebook import tqdm
import os, re
import glob
import calendar

import pendulum

In [ ]:
# 한글 폰트 사용
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[2]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Nanum%20Gothic"
change_matplotlib_font(font_download_url)
# 마이너스 폰트 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

## Data Preprocessing

### Load Data

In [ ]:
path = '/content/drive/MyDrive/구내식당/water/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
holiday = pd.read_csv(path+'holidays.csv', index_col=0)
corona = pd.read_csv(path+'corona_data.csv')

df = pd.concat([train, test])
df = df.fillna(0)
df.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']
df = df.drop(columns=['조식', '중식','석식'])

In [ ]:
df['일자'] = df['일자'].astype('datetime64')
corona['일자'] = corona['일자'].astype('datetime64')
df['출근인원'] = df['정원']-(df['휴가자']+df['출장자']+df['재택근무자'])
# df['휴가비율'] = df['휴가자']/df['정원']
# df['출장비율'] = df['출장자']/df['정원']
# df['야근비율'] = df['야근자']/df['출근인원']
# df['재택비율'] = df['재택근무자']/df['정원']
# df['휴가_출장'] = df['휴가자']+df['출장자']

## 코로나
* 재택근무자가 발생한 2020-01-06 이전과 이후의 차이가 크다

In [ ]:
train = df.iloc[:train.shape[0], :]
test = df.iloc[train.shape[0]:, :]

coro_ind = train[train['재택근무자']>0].index[0]
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(121)
sns.heatmap(train.iloc[coro_ind:, :].drop(columns='재택근무자').corr(),
            annot=True, fmt='.2f',
            cmap='RdYlBu_r',
            vmin=-1, vmax=1
            )
plt.title('코로나 이후 식수와 타변수의 상관관계')

plt.subplot(122)
sns.heatmap(train.iloc[:coro_ind, :].drop(columns='재택근무자').corr(),
            annot=True, fmt='.2f',
            cmap='RdYlBu_r',
            vmin=-1, vmax=1
            )
plt.title('코로나 이전 식수와 타변수의 상관관계')


plt.show()

### 년, 월, 일 컬럼 생성

In [ ]:
df['년'] = df['일자'].dt.year
df['월'] = df['일자'].dt.month
df['일'] = df['일자'].dt.day
month_to_season = {1: 3,2: 3,3:0,4:0,5:0,6:1,7:1,8:1,9:2,10:2,11:2,12: 3}
df['계절'] = df['월'].apply(lambda x : month_to_season[x])

df['요일'] = df['일자'].dt.weekday
df['야근_가능'] = df['요일'].apply(lambda x : 1 if (x==2) or (x==4) else 0)

In [ ]:
import pendulum

train = df.iloc[:train.shape[0], :]
test = df.iloc[train.shape[0]:, :]
train['주차'] = train['일자'].apply(lambda x: pendulum.parse(str(x)).week_of_month)
test['주차'] = test['일자'].apply(lambda x: pendulum.parse(str(x)).week_of_month)

repair_2017 = train[(train['년']==2017)&(train['주차']<0)]['일자'].dt.week
repair_2021 = train[(train['년']==2021)&(train['주차']<0)]['일자'].dt.week
test_repair = test[(test['년']==2021)&(test['주차']<0)]['일자'].dt.week

train['주차'][list(repair_2017.index)] = repair_2017.values
train['주차'][list(repair_2021.index)] = repair_2021.values
test['주차'][list(test_repair.index)] = test_repair.values
train['주차'][list(train[train['주차']==-46].index)] = np.array([6, 6, 6])

df = pd.concat([train, test])
df['주차'] = df['주차'].apply(lambda x: 5 if x==6 else x)
df['연_주차']= df['일자'].dt.weekofyear

In [ ]:
df['중식계/출근인원'] = df['중식계']/df['출근인원']
df['석식계/출근인원'] = df['석식계']/df['출근인원']

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(141)
plt.plot(df.groupby(['년']).mean()['중식계/출근인원'][:-1])
plt.plot(df.groupby(['년']).mean()['석식계/출근인원'][:-1])
plt.xticks(df.groupby(['년']).sum().index[:-1])
plt.xlabel('년도')
plt.ylabel('출근인원 대비 식수인원')
plt.title('년도별 출근인원 대비 식수인원 추이')

plt.subplot(142)
plt.plot(df.groupby(['월']).mean()['중식계/출근인원'])
plt.plot(df.groupby(['월']).mean()['석식계/출근인원'])
plt.xlabel('월별')
plt.ylabel('출근인원 대비 식수인원')
plt.title('월별 출근인원 대비 식수인원 추이')

plt.subplot(143)
plt.plot(df.groupby(['일']).mean()['중식계/출근인원'])
plt.plot(df.groupby(['일']).mean()['석식계/출근인원'])
plt.xticks(rotation=20)
plt.xlabel('일별')
plt.ylabel('출근인원 대비 식수인원')
plt.title('일별 출근인원 대비 식수인원 추이')
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(141)
plt.plot(df.groupby(['주차']).mean()['중식계/출근인원'])
plt.plot(df.groupby(['주차']).mean()['석식계/출근인원'])
plt.xlabel('주차')
plt.ylabel('출근인원 대비 식수인원')
plt.title('주차별 출근인원 대비 식수인원 추이')

plt.subplot(142)
plt.plot(df.groupby(['계절']).mean()['중식계/출근인원'])
plt.plot(df.groupby(['계절']).mean()['석식계/출근인원'])
plt.xlabel('계절별')
plt.ylabel('출근인원 대비 식수인원')
plt.title('계절별 출근인원 대비 식수인원 추이')

plt.subplot(143)
plt.plot(df.groupby(['연_주차']).mean()['중식계/출근인원'])
plt.plot(df.groupby(['연_주차']).mean()['석식계/출근인원'])
plt.xticks(rotation=20)
plt.xlabel('연_주차별')
plt.ylabel('출근인원 대비 식수인원')
plt.title('연_주차별 출근인원 대비 식수인원 추이')
plt.show()

### 요일

In [ ]:
df.groupby(['요일']).mean()

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.plot(df[df['요일']==4]['일자'], df[df['요일']==4]['야근자'])
plt.xlabel('년도')
plt.ylabel('야근자')
plt.title('금요일_야근자_추이')

plt.subplot(122)
plt.plot(df[df['요일']==2]['일자'], df[df['요일']==2]['야근자'])
plt.xlabel('년도')
plt.ylabel('야근자')
plt.title('수요일_야근자_추이')
plt.show()

## 공휴일 인사이트

### 공휴일 데이터 추가
* Only_공휴일 전후 컬럼
* 공휴일+휴일 전후 컬럼 추가

In [ ]:
holiday = holiday.reset_index()
holiday['date'] = pd.to_datetime(holiday['date'])

df['only_before_holiday'] = df['일자'].apply(lambda x : 1 if (x+dt.timedelta(1) in holiday['date'].tolist()) else 0)
df['only_after_holiday'] = df['일자'].apply(lambda x : 1 if (x-dt.timedelta(1) in holiday['date'].tolist()) else 0)

df['before_holiday'] = df['일자'].apply(lambda x : 1 if (x+dt.timedelta(1) in holiday['date'].tolist())\
                                      or ((x+dt.timedelta(1)).weekday() == 5) or ((x+dt.timedelta(1)).weekday() == 6) else 0)
df['after_holiday'] = df['일자'].apply(lambda x : 1 if (x-dt.timedelta(1) in holiday['date'].tolist())\
                                     or ((x-dt.timedelta(1)).weekday() == 5) or ((x-dt.timedelta(1)).weekday() == 6) else 0)

# train = pd.concat([df.iloc[:train.shape[0],:], target_df], axis=1)

### 공휴일 전후로 데이터의 흐름

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(131)
plt.boxplot([df[df['only_before_holiday']==1]['휴가자'], df[df['only_after_holiday']==1]['휴가자'],\
             df[(df['only_before_holiday']==1)&(df['only_after_holiday']==1)]['휴가자'], df[(df['only_before_holiday']==0)&(df['only_after_holiday']==0)]['휴가자']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('휴가자 수 평균')
plt.title('휴가와-쉬는날 연관성')

plt.subplot(132)
plt.boxplot([df[df['only_before_holiday']==1]['중식계'], df[df['only_after_holiday']==1]['중식계'],\
             df[(df['only_before_holiday']==1)&(df['only_after_holiday']==1)]['중식계'], df[(df['only_before_holiday']==0)&(df['only_after_holiday']==0)]['중식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('중식계 수 평균')
plt.title('중식계-쉬는날 연관성')

plt.subplot(133)
plt.boxplot([df[df['only_before_holiday']==1]['석식계'], df[df['only_after_holiday']==1]['석식계'],\
             df[(df['only_before_holiday']==1)&(df['only_after_holiday']==1)]['석식계'], df[(df['only_before_holiday']==0)&(df['only_after_holiday']==0)]['석식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('석식계 수 평균')
plt.title('석식계-쉬는날 연관성')

plt.show()

### 휴일 전후로 데이터의 흐름

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(131)
plt.boxplot([df[df['before_holiday']==1]['휴가자'], df[df['after_holiday']==1]['휴가자'],\
             df[(df['before_holiday']==1)&(df['after_holiday']==1)]['휴가자'], df[(df['before_holiday']==0)&(df['after_holiday']==0)]['휴가자']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('휴가자 수 평균')
plt.title('휴가와-쉬는날 연관성')

plt.subplot(132)
plt.boxplot([df[df['before_holiday']==1]['중식계'], df[df['after_holiday']==1]['중식계'],\
             df[(df['before_holiday']==1)&(df['after_holiday']==1)]['중식계'], df[(df['before_holiday']==0)&(df['after_holiday']==0)]['중식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('중식계 수 평균')
plt.title('중식계-쉬는날 연관성')

plt.subplot(133)
plt.boxplot([df[df['before_holiday']==1]['석식계'], df[df['after_holiday']==1]['석식계'],\
             df[(df['before_holiday']==1)&(df['after_holiday']==1)]['석식계'], df[(df['before_holiday']==0)&(df['after_holiday']==0)]['석식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('석식계 수 평균')
plt.title('석식계-쉬는날 연관성')

plt.show()

## 요일 인사이트

### 야근인원
* 수요일, 금요일은 야근 인원이 현저히 적다.
    * 야근 금지 -> 석식계와 큰 연관성
    * 월화목 vs 수금으로 구분

* 석식계가 0인 데이터는 어떻게 처리할 것인가?

In [ ]:
bar_width = 0.35
label = ['금', '목' ,'수', '월', '화']
index = np.arange(train['요일'].nunique())
alpha = 0.5
plt.figure(figsize=(12, 8))

plt.bar(index, train.groupby(['요일'])['야근자'].sum(), label='야근', width=0.4)
plt.bar(index+bar_width, train.groupby(['요일'])['석식계'].sum(), label='석식', width=0.4)
plt.xticks(index, label)
plt.legend('best', labels=['야근', '석식'])
plt.title('요일 별 야근 인원', fontsize=12)
plt.show()

### 출장자
* 요일별로 출장자의 차이가 크지는 않음
    * 월요일보다 금요일이 조금 많은 정도
    * 하지만 출장자와 중,석식은 관련이 있음


In [ ]:
plt.bar(train.groupby(['요일'])['출장자'].sum().index, train.groupby(['요일'])['출장자'].sum())
plt.title('요일 별 출장 인원', fontsize=12)
plt.show()

### 휴가자
* 월요일과 금요일이 훨씬 많음
    * 휴가는 붙여서 사용 가능
        * 당일 사용 가능
    * 공휴일과도 연관성이 짙을 것
    * 날씨와도 연관성이 있지 않을까

In [ ]:
plt.bar(train.groupby(['요일'])['휴가자'].sum().index, train.groupby(['요일'])['휴가자'].sum())
plt.title('요일 별 휴가 인원', fontsize=12)
plt.show()

## 재택 근무자
* 코로나 검사와 높은 연관성이 있는 것으로 추정
    * 검사를 받을 경우 재택이라고 함

In [ ]:
plt.plot(train['일자'].iloc[800:], train['재택근무자'].iloc[800:])
plt.plot(train['일자'].iloc[800:], train['중식계'].iloc[800:])
plt.xticks(rotation=20)
plt.title('재택 근무자 추이')

plt.show()

In [ ]:
train_18 = train.loc[train['년']==2018]
train_19 = train.loc[train['년']==2019]
train_20 = train.loc[train['년']==2020]

In [ ]:
plt.plot(train.groupby(['년'])['정원'].mean())
plt.title('년도 별 정원', fontsize=12)

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
bar_width = 0.3
index = train_19.groupby(['월'])['중식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['중식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['중식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['중식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['중식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('중식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 중식계에 끼친 영향', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
bar_width = 0.3
index = train_19.groupby(['월'])['석식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['석식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['석식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['석식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['석식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('석식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 석식계에 끼친 영향', fontsize=12)
plt.show()